In [1]:
import datetime
import stock_marker_binance
import model_design
import pandas as pd
from sklearn.utils import class_weight
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

In [2]:
EPOCHS =  2000
TIME_FRAME = [2, 3, 4, 5]
TRAIN_FRACTION = 0.75

In [3]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [4]:
def smote(data, train_frac):
    '''
    Oversample the minority class using SMOTE prior to OHE.
    '''

    n_train = int(data.shape[0] * train_frac)
    n_test = data.shape[0] - n_train

    train_data = data.iloc[:n_train, :]
    test_data = data.iloc[n_train:, :]

    x_train = train_data.drop(columns = ['action'])
    y_train = train_data['action']

    x_test = test_data.drop(columns = ['action'])
    y_test = test_data['action']

    smote_inst = SMOTE(sampling_strategy='auto')
    x_train , y_train = smote_inst.fit_resample(x_train, y_train)
    
    train_data = pd.concat([x_train, y_train], axis=1)
    test_data = pd.concat([x_test, y_test], axis=1)

    data = pd.concat([train_data, test_data], axis=0)

    return data, n_test

In [5]:
def OneHotEncoding(df_result):
    enc = OneHotEncoder()

    OHE_array = enc.fit_transform(df_result[['action']]).toarray()

    class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                    classes=np.unique(df_result['action']),
                                                    y=df_result['action'])



    class_weights_dict = dict(zip(enc.categories_[0], class_weights))

    print(class_weights)
    print(class_weights_dict.keys())

    
    OHE_df = pd.DataFrame(OHE_array, columns=enc.categories_[0])
    df_result = pd.merge(df_result, OHE_df, left_index=True, right_index=True)
    df_result.drop(columns=['action'], inplace=True)
    return df_result, class_weights_dict

In [6]:
def get_train_test_sets(data, n_test):
    
    x_train = data[:-n_test, :, :-4]
    y_train = data[:-n_test, -1:, -4:].reshape(-1,4)
    x_test = data[-n_test:, :, :-4]
    y_test = data[-n_test:, -1:, -4:].reshape(-1,4)
    return x_train, y_train, x_test, y_test

In [7]:
# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [8]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [9]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1577840399999,1.000930e+06,2504
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1577843999999,1.474278e+06,4885
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1577847599999,9.940256e+05,3046
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1577851199999,6.473610e+05,2818
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1577854799999,4.430067e+05,2264


In [10]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year


In [11]:
df_result.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1577840399999,1.000930e+06,2504,0.000000,1.000000e+00,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1577843999999,1.474278e+06,4885,0.258819,9.659258e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1577847599999,9.940256e+05,3046,0.500000,8.660254e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1577851199999,6.473610e+05,2818,0.707107,7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1577854799999,4.430067e+05,2264,0.866025,5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
5,2020-01-01 05:00:00,130.20,130.47,130.11,130.30,4243.60640,1577858399999,5.529727e+05,2426,0.965926,2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
6,2020-01-01 06:00:00,130.31,130.75,130.26,130.44,3668.90166,1577861999999,4.789442e+05,2347,1.000000,6.123234e-17,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
7,2020-01-01 07:00:00,130.47,130.71,130.14,130.24,4147.17413,1577865599999,5.407707e+05,2568,0.965926,-2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
8,2020-01-01 08:00:00,130.24,130.41,129.87,130.36,7541.44497,1577869199999,9.808940e+05,3039,0.866025,-5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
9,2020-01-01 09:00:00,130.40,130.62,130.13,130.17,4808.20496,1577872799999,6.268190e+05,2696,0.707107,-7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020


In [12]:

df_result = stock_marker_binance.std(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding STDs")

df_result = stock_marker_binance.adding_MAs(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding MAs")


df_result = stock_marker_binance.adding_ratio(df_result, ['close', 'quote_asset_volume', 'num_trades'], ['MA'])
df_result = stock_marker_binance.adding_ratio(df_result, ['close'], ['STD'])
print("Complete adding ratios")

df_result = stock_marker_binance.rsi(df_result, ['close', 
                                                #  'quote_asset_volume', 'num_trades'
                                                 ])
print("Complete adding RSI")

df_result = stock_marker_binance.marking_buy_sell_actions(df_result)
print("Complete marking buy sell actions")

df_result = stock_marker_binance.marking_adjacent_actions(df_result)
print("Complete marking adjacent actions")

df_result = stock_marker_binance.marking_hold_wait_actions(df_result)
print("Complete marking hold wait actions")

df_result = df_result.drop(columns=['timestamp','open', 'high', 'low' ,'close_time', 'volume',
                                    'close',  'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)
# df_result = stock_marker_binance.renaming_cols(df_result, STOCK_TICKER + "_", ['timestamp', 'action'])

Complete adding STDs
Complete adding MAs
Complete adding ratios
Complete adding RSI
Complete marking buy sell actions
Complete marking adjacent actions
Complete marking hold wait actions


In [13]:
df_result.groupby('action').count()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,STD3_close,...,RATIO_close_and_STD192,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close
action,,,,,,,,,,,,,,,,,,,,,
B,2341,2341,2341,2341,2341,2341,2341,2341,2341,2341,...,2341,2341,2341,2341,2341,2341,2341,2341,2341,2341
H,11121,11121,11121,11121,11121,11121,11121,11121,11121,11121,...,11121,11121,11121,11121,11121,11121,11121,11121,11121,11121
S,2650,2650,2650,2650,2650,2650,2650,2650,2650,2650,...,2650,2650,2650,2650,2650,2650,2650,2650,2650,2650
W,18832,18832,18832,18832,18832,18832,18832,18832,18832,18832,...,18832,18832,18832,18832,18832,18832,18832,18832,18832,18832


In [14]:
# Dropping cols
drop_cols_list =[]
for col in df_result.columns:
    if col[:2]=="MA" or col[:3]=="STD":
        drop_cols_list.append(col)
df_result.drop(columns=drop_cols_list, inplace=True)

In [15]:
df_result

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close,action
0,0.000000,1.000000e+00,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,15.673192,65.600000,53.818828,56.451613,49.606299,45.735591,58.575993,56.921027,56.004265,B
1,0.258819,9.659258e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,15.709274,72.784810,59.938367,66.561514,61.406250,48.108449,58.861391,56.921027,55.899593,S
2,0.500000,8.660254e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,15.713150,81.465517,51.035782,67.384615,63.572149,48.991400,60.461694,57.310652,55.990641,W
3,0.707107,7.071068e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,15.599209,78.099174,73.841962,63.047809,63.284782,52.077793,60.000000,57.287449,56.013769,W
4,0.866025,5.000000e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,15.667762,81.468531,78.378378,71.428571,65.007180,55.126699,60.382340,57.649960,56.242723,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34939,0.707107,7.071068e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.999538,...,42.424458,63.647491,14.575412,62.740044,72.020706,57.083589,54.378381,51.021152,49.886262,W
34940,0.866025,5.000000e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,1.001322,...,42.530161,74.787776,25.061152,66.604418,72.069594,57.045402,54.645727,51.030496,50.098711,W
34941,0.965926,2.588190e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.999021,...,42.404276,39.237233,25.491970,50.822577,70.374137,56.957313,54.037629,50.948907,50.349035,W
34942,1.000000,6.123234e-17,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.997117,...,42.276558,31.499741,41.071429,32.320816,66.723177,55.541223,53.031366,50.579826,50.167989,W


In [16]:
df_result, n_test = smote(df_result, TRAIN_FRACTION)

In [17]:
df_result, class_weights_dict = OneHotEncoding(df_result)

[1.13331879 0.9716887  1.13645978 0.82742141]
dict_keys(['B', 'H', 'S', 'W'])


In [19]:
df_result.head()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close,B,H,S,W
0,0.000000,1.000000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,56.451613,49.606299,45.735591,58.575993,56.921027,56.004265,1.0,0.0,0.0,0.0
1,0.258819,0.965926,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,66.561514,61.406250,48.108449,58.861391,56.921027,55.899593,0.0,0.0,1.0,0.0
2,0.500000,0.866025,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,67.384615,63.572149,48.991400,60.461694,57.310652,55.990641,0.0,0.0,0.0,1.0
3,0.707107,0.707107,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,63.047809,63.284782,52.077793,60.000000,57.287449,56.013769,0.0,0.0,0.0,1.0
4,0.866025,0.500000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,71.428571,65.007180,55.126699,60.382340,57.649960,56.242723,0.0,0.0,0.0,1.0


In [20]:
df_result.columns

Index(['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'mon_sin', 'mon_cos',
       'weekday_sin', 'weekday_cos', 'year', 'RATIO_close_and_MA3',
       'RATIO_close_and_MA6', 'RATIO_close_and_MA12', 'RATIO_close_and_MA24',
       'RATIO_close_and_MA48', 'RATIO_close_and_MA96', 'RATIO_close_and_MA192',
       'RATIO_close_and_MA384', 'RATIO_quote_asset_volume_and_MA3',
       'RATIO_quote_asset_volume_and_MA6', 'RATIO_quote_asset_volume_and_MA12',
       'RATIO_quote_asset_volume_and_MA24',
       'RATIO_quote_asset_volume_and_MA48',
       'RATIO_quote_asset_volume_and_MA96',
       'RATIO_quote_asset_volume_and_MA192',
       'RATIO_quote_asset_volume_and_MA384', 'RATIO_num_trades_and_MA3',
       'RATIO_num_trades_and_MA6', 'RATIO_num_trades_and_MA12',
       'RATIO_num_trades_and_MA24', 'RATIO_num_trades_and_MA48',
       'RATIO_num_trades_and_MA96', 'RATIO_num_trades_and_MA192',
       'RATIO_num_trades_and_MA384', 'RATIO_close_and_STD3',
       'RATIO_close_and_STD6', 'RATIO_close_a

In [21]:
max_val_PR = 0.65

for time_step in TIME_FRAME:
    df_tensor = dataframe_to_tensor(df_result, time_step)
    x_train, y_train, x_test, y_test = get_train_test_sets(df_tensor, n_test)
    
    df_tensor.shape

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    model, history = model_design.model_design(EPOCHS,  x_train, y_train, x_test, y_test, class_weights_dict)

    val_PR = round(max(history.history['val_PR']),3)
    
    x = datetime.datetime.now()
    
    model_name = "models/" + x.strftime("%d-%m-%Y %H-%M-%S") +" val_PR " + str(val_PR) + " time frame " + str(time_step) + ".hd5"

    if val_PR > max_val_PR:
        max_val_PR = val_PR
        model.save(model_name,  save_format = 'h5')
    
    print(model_name)
    print(val_PR)
    print(150 * "-")


(53591, 2, 49)
(53591, 4)
(8736, 2, 49)
(8736, 4)





C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


models/02-02-2024 22-07-50 val_PR 0.813 time frame 2.hd5
0.813
------------------------------------------------------------------------------------------------------------------------------------------------------
(53590, 3, 49)
(53590, 4)
(8736, 3, 49)
(8736, 4)
models/02-02-2024 22-12-58 val_PR 0.994 time frame 3.hd5
0.994
------------------------------------------------------------------------------------------------------------------------------------------------------
(53589, 4, 49)
(53589, 4)
(8736, 4, 49)
(8736, 4)
models/02-02-2024 22-20-22 val_PR 0.982 time frame 4.hd5
0.982
------------------------------------------------------------------------------------------------------------------------------------------------------
(53588, 5, 49)
(53588, 4)
(8736, 5, 49)
(8736, 4)
models/02-02-2024 22-36-35 val_PR 0.933 time frame 5.hd5
0.933
-------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 5, 49)]           0         
                                                                 
 normalization_3 (Normaliza  (None, 5, 49)             99        
 tion)                                                           
                                                                 
 lstm_6 (LSTM)               (None, 5, 49)             19404     
                                                                 
 lstm_7 (LSTM)               (None, 6)                 1344      
                                                                 
 dense_6 (Dense)             (None, 6)                 42        
                                                                 
 dense_7 (Dense)             (None, 4)                 28        
                                                           